In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
import statistics as stats
import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
import yaml
from tkinter import *
import tkinter
from geopy.geocoders import Nominatim
from geopy import distance
from geopy import Point

## Dataset files 

In [3]:
#reviewJson = "..\\data\\Export_CleanedReview.json"
reviewJsonWithClosedRes = "..\\data\\Export_CleanedReviewWithClosedRes.json"
reviewJsonToronto = "..\\data\\Export_TorontoData.json"


## Load Data
Select top frenquent user and top frequenty restaurants that had at least 1 review >= 4 stars (Kickking out users that gave all  reviews <=3 and restaurants that never got start >= 4 stars)

In [4]:
def get_yelp_df(path = 'data/', filename = 'Export_CleanedReview.json', sampling=False, top_user_num=6100, top_item_num=4000):
    """
    Get the pandas dataframe
    Sampling only the top users/items by density 
    Implicit representation applies
    """
    with open(filename,'r') as f:
        data = f.readlines()
        data = list(map(json.loads, data))
    
    data = data[0]
    #Get all the data from the dggeata file
    df = pd.DataFrame(data)
    
    df.rename(columns={'stars': 'review_stars', 'text': 'review_text', 'cool': 'review_cool',
                       'funny': 'review_funny', 'useful': 'review_useful'},
              inplace=True)

    df['business_num_id'] = df.business_id.astype('category').\
        cat.rename_categories(range(0, df.business_id.nunique()))
    df['business_num_id'] = df['business_num_id'].astype('int')

    df['user_num_id'] = df.user_id.astype('category').\
    cat.rename_categories(range(0, df.user_id.nunique()))
    df['user_num_id'] = df['user_num_id'].astype('int')

    df['timestamp'] = df['date'].apply(date_to_timestamp)

    if sampling:
        df = filter_yelp_df(df, top_user_num=top_user_num, top_item_num=top_item_num)
        # Refresh num id
        df['business_num_id'] = df.business_id.astype('category').\
        cat.rename_categories(range(0, df.business_id.nunique()))
        df['business_num_id'] = df['business_num_id'].astype('int')
        
        df['user_num_id'] = df.user_id.astype('category').\
        cat.rename_categories(range(0, df.user_id.nunique()))
        df['user_num_id'] = df['user_num_id'].astype('int')
#     drop_list = ['date','review_id','review_funny','review_cool','review_useful']
#     df = df.drop(drop_list, axis=1)

    df = df.reset_index(drop = True)

    return df 

def filter_yelp_df(df, top_user_num=6100, top_item_num=4000):
    #Getting the reviews where starts are above 3
    df_implicit = df[df['review_stars']>3]
    frequent_user_id = df_implicit['user_num_id'].value_counts().head(top_user_num).index.values
    frequent_item_id = df_implicit['business_num_id'].value_counts().head(top_item_num).index.values
    return df.loc[(df['user_num_id'].isin(frequent_user_id)) & (df['business_num_id'].isin(frequent_item_id))]

def date_to_timestamp(date):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return time.mktime(dt.timetuple())

def df_to_sparse(df, row_name='userId', col_name='movieId', value_name='rating',
                 shape=None):
    rows = df[row_name]
    cols = df[col_name]
    if value_name is not None:
        values = df[value_name]
    else:
        values = [1]*len(rows)

    return csr_matrix((values, (rows, cols)), shape=shape)

## Get rating-UI and timestamp-UI matrix from original df

In [5]:
def get_rating_timestamp_matrix(df, sampling=False, top_user_num=6100, top_item_num=4000):
    """
    """
    #make the df implicit with top frenquent users and 
    #no need to sample anymore if df was sampled before 
    if sampling:
        df = filter_yelp_df(df, top_user_num=top_user_num, top_item_num=top_item_num)

    rating_matrix = df_to_sparse(df, row_name='user_num_id',
                                 col_name='business_num_id',
                                 value_name='review_stars',
                                 shape=None)
    
    #Have same dimension and data entries with rating_matrix, except that the review stars are - user avg
#     ratingWuserAvg_matrix = df_to_sparse(df, row_name='user_num_id',
#                                  col_name='business_num_id',
#                                  value_name='reviewStars_userAvg',
#                                  shape=None)
    
    timestamp_matrix = df_to_sparse(df, row_name='user_num_id',
                                    col_name='business_num_id',
                                    value_name='timestamp',
                                    shape=None)
    
    
    IC_matrix = get_I_C(df)
#     ratingWuserAvg_matrix
    return rating_matrix, timestamp_matrix, IC_matrix

In [6]:
def get_I_C(df):
    lst = df.categories.values.tolist()
    cat = []
    for i in range(len(lst)):
        cat.extend(lst[i].split(', '))
    unique_cat = set(cat)
    #     set categories id
    df_cat = pd.DataFrame(list(unique_cat),columns=["Categories"])
    df_cat['cat_id'] = df_cat.Categories.astype('category').cat.rename_categories(range(0, df_cat.Categories.nunique()))
    dict_cat = df_cat.set_index('Categories')['cat_id'].to_dict()
    
    df_I_C = pd.DataFrame(columns=['business_num_id', 'cat_id'])
    
    for i in range((df['business_num_id'].unique().shape)[0]):
        df_temp = df[df['business_num_id'] == i].iloc[:1]
        temp_lst = df_temp['categories'].to_list()[0].split(",")
        for j in range(len(temp_lst)):
            df_I_C = df_I_C.append({'business_num_id' : i  , 'cat_id' : dict_cat[temp_lst[j].strip()]} , ignore_index=True)
    
    IC_Matrix = df_to_sparse(df_I_C, row_name='business_num_id',
                                 col_name='cat_id',
                                 value_name=None,
                                 shape=None)    
    return IC_Matrix

## Time ordered split 

In [7]:
def time_ordered_split(rating_matrix, ratingWuserAvg_matrix, timestamp_matrix, ratio=[0.5, 0.2, 0.3],
                       implicit=True, remove_empty=False, threshold=3,
                       sampling=False, sampling_ratio=0.1):
    """
    Split the data to train,valid,test by time
    ratio:  train:valid:test
    threshold: for implicit representation
    """
    if implicit:
        temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
        temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
        rating_matrix = temp_rating_matrix
        timestamp_matrix = timestamp_matrix.multiply(rating_matrix)
        #ratingWuserAvg_matrix = ratingWuserAvg_matrix.multiply(rating_matrix)

    nonzero_index = None

    #Default false, not removing empty columns and rows
    #Should not have this case, since users should have at least 1 record of 4,5 
    #And restuarant should have at least 1 record of 4,5 
    if remove_empty:
        # Remove empty columns. record original item index
        nonzero_index = np.unique(rating_matrix.nonzero()[1])
        rating_matrix = rating_matrix[:, nonzero_index]
        timestamp_matrix = timestamp_matrix[:, nonzero_index]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[:, nonzero_index]

        # Remove empty rows. record original user index
        nonzero_rows = np.unique(rating_matrix.nonzero()[0])
        rating_matrix = rating_matrix[nonzero_rows]
        timestamp_matrix = timestamp_matrix[nonzero_rows]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[nonzero_rows]

    user_num, item_num = rating_matrix.shape

    rtrain = []
    rtrain_userAvg = []
    rtime = []
    rvalid = []
    rvalid_userAvg = []
    rtest = []
    rtest_userAvg = []
    # Get the index list corresponding to item for train,valid,test
    item_idx_train = []
    item_idx_valid = []
    item_idx_test = []
    
    for i in tqdm(range(user_num)):
        #Get the non_zero indexs, restuarants where the user visited/liked if implicit 
        item_indexes = rating_matrix[i].nonzero()[1]
        
        #Get the data for the user
        data = rating_matrix[i].data
        
        #Get time stamp value 
        timestamp = timestamp_matrix[i].data
        
        #Get review stars with user avg data 
        if implicit == False:
            dataWuserAvg = ratingWuserAvg_matrix[i].data
        
        #Non zero reviews for this user
        num_nonzeros = len(item_indexes)
        
        #If the user has at least one review
        if num_nonzeros >= 1:
            #Get number of test and valid data 
            #train is 30%
            num_test = int(num_nonzeros * ratio[2])
            #validate is 50%
            num_valid = int(num_nonzeros * (ratio[1] + ratio[2]))

            valid_offset = num_nonzeros - num_valid
            test_offset = num_nonzeros - num_test

            #Sort the timestamp for each review for the user
            argsort = np.argsort(timestamp)
            
            #Sort the reviews for the user according to the time stamp 
            data = data[argsort]
            
            #Sort the review with user avg accoridng to the time stamp
            if implicit == False:
                dataWuserAvg = dataWuserAvg[argsort]
            
            #Non-zero review index sort according to time
            item_indexes = item_indexes[argsort]
            
            #list of ratings, num of valid_offset index, index where there's non-zeros
            rtrain.append([data[:valid_offset], np.full(valid_offset, i), item_indexes[:valid_offset]])
            
            if implicit == False:
                #Changing valid set to binary
                count=valid_offset
                for eachData in data[valid_offset:test_offset]:
                    #if rating-avgRating > 0 then like
                    if eachData >= 4:
                        data[count] = 1
                    else:
                        data[count] = 0
                    count += 1
                
            #50%-70%
            rvalid.append([data[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                           item_indexes[valid_offset:test_offset]])
            #remaining 30%
            rtest.append([data[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
            
            if implicit == False:
                #Now for the rating matrix that considers user average rating
                #list of ratings, num of valid_offset index, index where there's non-zeros
                rtrain_userAvg.append([dataWuserAvg[:valid_offset], np.full(valid_offset, i), item_indexes[:valid_offset]])
                #50%-70%

                #Changing valid set to binary
                count=valid_offset
                for eachData in dataWuserAvg[valid_offset:test_offset]:
                    #if rating-avgRating > 0 then like
                    if eachData > 0:
                        dataWuserAvg[count] = 1
                    else:
                        dataWuserAvg[count] = 0
                    count += 1

                rvalid_userAvg.append([dataWuserAvg[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                               item_indexes[valid_offset:test_offset]])

                #Change test set to binary even we don't use it
                countTest = test_offset
                for eachData in dataWuserAvg[test_offset:]:
                    #if rating-avgRating > 0 then like
                    if eachData > 0:
                        dataWuserAvg[count] = 1
                    else:
                        dataWuserAvg[count] = 0
                    count += 1


                #remaining 30%
                rtest_userAvg.append([dataWuserAvg[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
                
            item_idx_train.append(item_indexes[:valid_offset])
            
#             item_idx_valid.append(item_indexes[valid_offset:test_offset])
#             item_idx_test.append(item_indexes[test_offset:])
        else:
            item_idx_train.append([])
#             item_idx_valid.append([])
#             item_idx_test.append([])
    
    rtrain = np.array(rtrain)
    rvalid = np.array(rvalid)
    rtest = np.array(rtest)
    if implicit == False:
        rtrain_userAvg = np.array(rtrain_userAvg)
        rvalid_userAvg = np.array(rvalid_userAvg)
        rtest_userAvg = np.array(rtest_userAvg)

    #take non-zeros values, row index, and column (non-zero) index and store into sparse matrix
    rtrain = sparse.csr_matrix((np.hstack(rtrain[:, 0]), (np.hstack(rtrain[:, 1]), np.hstack(rtrain[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rvalid = sparse.csr_matrix((np.hstack(rvalid[:, 0]), (np.hstack(rvalid[:, 1]), np.hstack(rvalid[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rtest = sparse.csr_matrix((np.hstack(rtest[:, 0]), (np.hstack(rtest[:, 1]), np.hstack(rtest[:, 2]))),
                              shape=rating_matrix.shape, dtype=np.float32)
    
    if implicit == False:
        rtrain_userAvg = sparse.csr_matrix((np.hstack(rtrain_userAvg[:, 0]), (np.hstack(rtrain_userAvg[:, 1]), np.hstack(rtrain_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rvalid_userAvg = sparse.csr_matrix((np.hstack(rvalid_userAvg[:, 0]), (np.hstack(rvalid_userAvg[:, 1]), np.hstack(rvalid_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rtest_userAvg = sparse.csr_matrix((np.hstack(rtest_userAvg[:, 0]), (np.hstack(rtest_userAvg[:, 1]), np.hstack(rtest_userAvg[:, 2]))),
                                  shape=rating_matrix.shape, dtype=np.float32)

    return rtrain, rvalid, rtest,rtrain_userAvg, rvalid_userAvg, rtest_userAvg, nonzero_index, timestamp_matrix, item_idx_train, item_idx_valid, item_idx_test

In [8]:
def time_ordered_splitModified(rating_matrix, ratingWuserAvg_matrix, timestamp_matrix, ratio=[0.5, 0.2, 0.3],
                       implicit=True, remove_empty=False, threshold=3,
                       sampling=False, sampling_ratio=0.1, trainSampling=1):
    """
    Split the data to train,valid,test by time
    ratio:  train:valid:test
    threshold: for implicit representation
    """
    
    
    if implicit:
        temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
        temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
        rating_matrix = temp_rating_matrix
        timestamp_matrix = timestamp_matrix.multiply(rating_matrix)
        #ratingWuserAvg_matrix = ratingWuserAvg_matrix.multiply(rating_matrix)

    nonzero_index = None

    #Default false, not removing empty columns and rows
    #Should not have this case, since users should have at least 1 record of 4,5 
    #And restuarant should have at least 1 record of 4,5 
    if remove_empty:
        # Remove empty columns. record original item index
        nonzero_index = np.unique(rating_matrix.nonzero()[1])
        rating_matrix = rating_matrix[:, nonzero_index]
        timestamp_matrix = timestamp_matrix[:, nonzero_index]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[:, nonzero_index]

        # Remove empty rows. record original user index
        nonzero_rows = np.unique(rating_matrix.nonzero()[0])
        rating_matrix = rating_matrix[nonzero_rows]
        timestamp_matrix = timestamp_matrix[nonzero_rows]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[nonzero_rows]

    user_num, item_num = rating_matrix.shape

    rtrain = []
    rtrain_userAvg = []
    rtime = []
    rvalid = []
    rvalid_userAvg = []
    rtest = []
    rtest_userAvg = []
    # Get the index list corresponding to item for train,valid,test
    item_idx_train = []
    item_idx_valid = []
    item_idx_test = []
    
    for i in tqdm(range(user_num)):
        #Get the non_zero indexs, restuarants where the user visited/liked if implicit 
        item_indexes = rating_matrix[i].nonzero()[1]        
        #Get the data for the user
        data = rating_matrix[i].data      
        #Get time stamp value 
        timestamp = timestamp_matrix[i].data 
        #Get review stars with user avg data 
        if implicit == False:
            dataWuserAvg = ratingWuserAvg_matrix[i].data

            
        #Non zero reviews for this user
        num_nonzeros = len(item_indexes)
        
        #If the user has at least one review
        if num_nonzeros >= 1:
            num_test = int(num_nonzeros * ratio[2])
            num_valid = int(num_nonzeros * (ratio[1] + ratio[2]))
            valid_offset = num_nonzeros - num_valid
            
            # Adding this for sampling for training set
            valid_offsetSample = int(valid_offset*trainSampling)
            test_offset = num_nonzeros - num_test
            
            #Sort the timestamp for each review for the user
            argsort = np.argsort(timestamp)
            
            #Sort the reviews for the user according to the time stamp 
            data = data[argsort]
            
            #Sort the review with user avg accoridng to the time stamp
            if implicit == False:
                dataWuserAvg = dataWuserAvg[argsort]
            
            #Non-zero review index sort according to time
            item_indexes = item_indexes[argsort]
            
            #list of ratings, num of valid_offset index, index where there's non-zeros
            #if take from old to new
            #rtrain.append([data[:valid_offsetSample], np.full(valid_offsetSample, i), item_indexes[:valid_offsetSample]])
            #if take from new to old
            rtrain.append([data[valid_offset-valid_offsetSample:valid_offset], np.full(valid_offsetSample, i), item_indexes[valid_offset-valid_offsetSample:valid_offset]])
            rvalid.append([data[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                           item_indexes[valid_offset:test_offset]])
            rtest.append([data[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
            
            if implicit == False:
                #Now for the rating matrix that considers user average rating
                #list of ratings, num of valid_offset index, index where there's non-zeros
                #from old to new
                #rtrain_userAvg.append([dataWuserAvg[:valid_offsetSample], np.full(valid_offsetSample, i), item_indexes[:valid_offsetSample]])
                #take nearest
                rtrain_userAvg.append([dataWuserAvg[valid_offset-valid_offsetSample:valid_offset], np.full(valid_offsetSample, i), item_indexes[valid_offset-valid_offsetSample:valid_offset]])                
                    
                rvalid_userAvg.append([dataWuserAvg[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                               item_indexes[valid_offset:test_offset]])
                
                rtest_userAvg.append([dataWuserAvg[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
                
            item_idx_train.append(item_indexes[:valid_offsetSample])
            
        else:
            item_idx_train.append([])
    
    rtrain = np.array(rtrain)
    rvalid = np.array(rvalid)
    rtest = np.array(rtest)
   
    if implicit == False:
        rtrain_userAvg = np.array(rtrain_userAvg)
        rvalid_userAvg = np.array(rvalid_userAvg)
        rtest_userAvg = np.array(rtest_userAvg)

    #take non-zeros values, row index, and column (non-zero) index and store into sparse matrix
    rtrain = sparse.csr_matrix((np.hstack(rtrain[:, 0]), (np.hstack(rtrain[:, 1]), np.hstack(rtrain[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rvalid = sparse.csr_matrix((np.hstack(rvalid[:, 0]), (np.hstack(rvalid[:, 1]), np.hstack(rvalid[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rtest = sparse.csr_matrix((np.hstack(rtest[:, 0]), (np.hstack(rtest[:, 1]), np.hstack(rtest[:, 2]))),
                              shape=rating_matrix.shape, dtype=np.float32)
    
    if implicit == False:
        rtrain_userAvg = sparse.csr_matrix((np.hstack(rtrain_userAvg[:, 0]), (np.hstack(rtrain_userAvg[:, 1]), np.hstack(rtrain_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rvalid_userAvg = sparse.csr_matrix((np.hstack(rvalid_userAvg[:, 0]), (np.hstack(rvalid_userAvg[:, 1]), np.hstack(rvalid_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rtest_userAvg = sparse.csr_matrix((np.hstack(rtest_userAvg[:, 0]), (np.hstack(rtest_userAvg[:, 1]), np.hstack(rtest_userAvg[:, 2]))),
                                  shape=rating_matrix.shape, dtype=np.float32)

    return rtrain, rvalid, rtest,rtrain_userAvg, rvalid_userAvg, rtest_userAvg, nonzero_index, timestamp_matrix, item_idx_train, item_idx_valid, item_idx_test


## Get df for training corpus

In [9]:
#Item idex matrix stores the reivews starts
#This function returns a list of index for the reviews included in training set 
def get_corpus_idx_list(df, item_idx_matrix):
    """
    Input: 
    df: total dataframe
    item_idx_matrix: train index list got from time_split 
    Output: row index in original dataframe for training data by time split
    """
    lst = []
    #For all the users: 5791
    for i in tqdm(range(len(item_idx_matrix))):
        
        #find row index where user_num_id is i
        a = df.index[df['user_num_id'] == i].tolist()
        
        #loop through the busienss id that the user i reviewed for in offvalid set 
        for item_idx in  item_idx_matrix[i]:
            
            #get the row index for reviews for business that the user liked in the train set
            b = df.index[df['business_num_id'] == item_idx].tolist()
            
            #Find the index for which this user liked, one user only rate a business once
            idx_to_add = list(set(a).intersection(b))
            
            if idx_to_add not in lst:
                lst.extend(idx_to_add)
    return lst

## Preprocess using Term Frequency - CounterVectorizer

In [10]:
#Stemming and Lemmatisation
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
# Get corpus and CountVector
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
nltk.download('stopwords')
lem = WordNetLemmatizer()
stem = PorterStemmer()
stop_words = set(stopwords.words("english"))
new_words = ['not_the']
stop_words = stop_words.union(new_words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\songya25\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\songya25\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [11]:
from tqdm import tqdm

#Should 'because' added?
def preprocess(df, reset_list = [',','.','?',';','however','but']):
    corpus = []
    for i in tqdm(range(df.shape[0])):
        text = df['review_text'][i]
        change_flg = 0
        #Convert to lowercase
        text = text.lower()
        
        ##Convert to list from string, loop through the review text
        text = text.split()
        
        #any sentence that encounters a not, the folloing words will become not phrase until hit the sentence end
        for j in range(len(text)):
            #Make the not_ hack
            if text[j] == 'not':
                change_flg = 1
#                 print 'changes is made after ', i
                continue
            #if was 1 was round and not hit a 'not' in this round
            if change_flg == 1 and any(reset in text[j] for reset in reset_list):
                text[j] = 'not_' + text[j]
                change_flg = 0
#                 print 'reset at ', i
            if change_flg == 1:
                text[j] = 'not_' + text[j]
        
        #Convert back to string
        text = " ".join(text)
        
        #Remove punctuations
#       text = re.sub('[^a-zA-Z]', ' ', text)
        
        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
        
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
        
        ##Convert to list from string
        text = text.split()
        
        ##Stemming
        ps=PorterStemmer()
        
        #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in  
                stop_words] 
        text = " ".join(text)
        corpus.append(text)
    return corpus

## All Model

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, X, row_name = 'business_num_id', binary = True, shape = (121994,6000)):
    """
    get the item-keyphrase matrix
    """
    rows = []
    cols = []
    vals = []
    #For each review history
    for i in tqdm(range(X.shape[0])):
        #Get the array of frequencies for document/review i 
        arr = X[i].toarray() 
        nonzero_element = arr.nonzero()[1]  # Get nonzero element in each line, keyphrase that appears index 
        length_of_nonzero = len(nonzero_element) #number of important keyphrase that appears
        
        # df[row_name][i] is the item idex
        #Get a list row index that indicates the document/review
        rows.extend(np.array([df[row_name][i]]*length_of_nonzero)) ## Item index
        #print(rows)
        
        #Get a list of column index indicating the key phrase that appears in i document/review
        cols.extend(nonzero_element) ## Keyword Index
        if binary:
            #Create a bunch of 1s
            vals.extend(np.array([1]*length_of_nonzero))
        else:
            #If not binary 
            vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


#Get a UI matrix if it's not item_similarity based or else IU
def predict(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
    
    #inverse to IU matrix
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    #for each user or item, depends UI or IU 
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        #Get prediction/similarity score for each user 1*num or user or num of items
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        #Decending accoding to similarity score, select top k
        similar_users = vector_u.argsort()[::-1][1:k+1]
        
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        
        #similar_users_weights_sum = np.sum(similar_users_weights)
        #print(similar_users_weights.shape)
        #shape: num of res * k
        similar_users_ratings = matrix_train[similar_users].toarray()
              
        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
        #print(prediction_scores_u)
        
        
        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
        
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    return res


#Preidction score is UI or IU?
def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    #for each user
    for user_index in tqdm(range(matrix_Train.shape[0])):
        
        #take the prediction scores for user 1 * num res
        vector_u = prediction_score[user_index]
        
        #The restuarant the user rated
        vector_train = matrix_Train[user_index]
        
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)

#topK: the number of restuarants we are suggesting 
#if vector_train has number, then the user has visited
def sub_routine(vector_u, vector_train, topK=500):

    #index where non-zero
    train_index = vector_train.nonzero()[1]
    
    vector_u = vector_u
    
    #get topk + num rated res prediction score descending, top index 
    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    
    #sort top prediction score index in range topK+len(train_index) into vector_u`
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    
    #deleted the rated res from the topk+train_index prediction score vector for user u 
    #Delete the user rated res index from the topk+numRated index
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    #so we only include the top K prediction score here
    return vector_u[:topK]

## Evaluation

In [13]:
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)


def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        #"R-Precision": r_precision,
        #"NDCG": ndcg,
        #"Clicks": click
    }

    local_metrics = {
        #"Precision": precisionk,
        #"Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))
        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = round(results[name],4)
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (round((np.average(results[name])),4),
                                                              round((1.96*np.std(results[name])/np.sqrt(num_users)),4))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))
    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = round(results[name],4)
    else:
        for name in global_metric_names:
            results_summary[name] = (round(np.average(results[name]),4), round((1.96*np.std(results[name])/np.sqrt(num_users)),4))
    output.update(results_summary)

    return output

## Add 2 matrices

In [14]:
## Utility functions
#3906 restuarant, 3000 keyphrase, 5791 user 
def add_two_matrix(ratio, U_I_matrix,I_K_matrix, shape = (3906, 3000+5791)):
    # ratio determine Keywords/User in the matrix
    rows = []
    cols = []
    datas = []
    I_U_matrix = U_I_matrix.transpose()
    
    #for each restuarant
    for i in tqdm(range(I_K_matrix.shape[0])):
        #key phrase that this item has, column(key phrase) index
        nonzero1 = I_K_matrix[i].nonzero()
        
        #user that rated this item, column(user) index 
        nonzero2 = I_U_matrix[i].nonzero()
        
        #Trying to create a sparse matrix that stores 
        #index of restuarant for (K + U) times
        row = [i]*(len(nonzero1[1])+len(nonzero2[1]))
        
        #column index for key phrase and users that are non-zero
        col = nonzero1[1].tolist()+ nonzero2[1].tolist()
        
        
        data = [ratio]*len(nonzero1[1])+[1-ratio]*len(nonzero2[1]) # Binary representation of I-K/U matrix
        
        rows.extend(row)
        cols.extend(col)
        datas.extend(data)
    return csr_matrix( (datas,(rows,cols)), shape=shape )

def transfer_to_implicit(rating_matrix, threshold = 0):
    temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
    temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
    rating_matrix = temp_rating_matrix
    return rating_matrix

In [15]:
def getImplicitMatrix(sparseMatrix, threashold=0):
    temp_matrix = sparse.csr_matrix(sparseMatrix.shape)
    temp_matrix[(sparseMatrix > threashold).nonzero()] = 1
    return temp_matrix

In [16]:
def predictUU(matrix_train, k, chooseWeigthMethod, similarity1=None, similarity2=None, similarity3=None, similarity4=None, similarity5=None, item_similarity_en = False):
    prediction_scores = []
    #Convert from list to ndarray, add an axis
    if isinstance(chooseWeigthMethod, list):
        chooseWeigthMethod = np.array(chooseWeigthMethod)[:, np.newaxis]
   
    "make sure that when passing in chooseWeightMethod, the weight must be aligned with similarity metrices, even if set to None"
    "They should add to 1 as well"
    #inverse to IU matrix
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    #for each user or item, depends UI or IU 
    for user_index in tqdm(range(matrix_train.shape[0])):
    #for user_index in tqdm(range(10,20)):
        
        numberSimilarMatrix = 0
        # Get user u's prediction scores for all items 
        #Get prediction/similarity score for each user 1*num or user or num of items
        if similarity1 is not None:
            vector_u1 = similarity1[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u1 = [0]*matrix_train.shape[0]
            
        if similarity2 is not None:
            vector_u2 = similarity2[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u2 = [0]*len(vector_u1)
            
        if similarity3 is not None:
            vector_u3 = similarity3[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u3 = [0]*len(vector_u1)
            
        if similarity4 is not None:
            vector_u4 = similarity4[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u4 = [0]*len(vector_u1)
        
        if similarity5 is not None:
            vector_u5 = similarity5[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u5 = [0]*len(vector_u1)
        
        #Temperary vector that stacks all 4 vectors together
        tempVector = np.array([vector_u1,vector_u2,vector_u3,vector_u4, vector_u5])
        
        if chooseWeigthMethod is None:
            #Get the similarity score from the first similarity matrix anyways 
            vector_u = vector_u1.copy()
            
        #If we are choosing the max, min, avg or similarity scores
        if chooseWeigthMethod is not None:
            if chooseWeigthMethod == 'max':
                vector_u = tempVector.max(axis=0)
            elif chooseWeigthMethod == 'min':
                vector_u = tempVector.min(axis=0)
            elif chooseWeigthMethod == 'average':
                vector_u = tempVector.mean(axis=0)
            elif isinstance(chooseWeigthMethod, np.ndarray):
                #Validate that number of weights passed in equals number of matrices
                #assert(len(chooseWeigthMethod) == numberSimilarMatrix)
                #Get the new combined similarity vector 
                weighted_u = tempVector * chooseWeigthMethod
                vector_u =np.sum(weighted_u, axis=0)
                #print((vector_u == vector_u4).sum())
                
        similar_users = vector_u.argsort()[::-1][1:k+1]
        
        # Get neighbors similarity weights and ratings
        #similar_users_weights = similarity1[user_index][similar_users]
        similar_users_weights = vector_u[similar_users]
        
        #similar_users_weights_sum = np.sum(similar_users_weights)
        #print(similar_users_weights.shape)
        #shape: num of res * k
        similar_users_ratings = matrix_train[similar_users].toarray()
              
        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
        #print(prediction_scores_u)
        
        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
        
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    return res

#### Get original dataframe out of the review datastet

In [17]:
df = get_yelp_df(path ='', filename=reviewJsonToronto, sampling= True)

#### Get rating-UI matrix and timestepm-UI matrix

In [18]:
rating_matrix, timestamp_matrix , I_C_matrix = get_rating_timestamp_matrix(df)

# get ratingWuserAvg_matrix
rating_array = rating_matrix.toarray()
user_average_array = rating_array.sum(axis = 1)/np.count_nonzero(rating_array,axis = 1)
init_UI = np.zeros(rating_array.shape)
init_UI[rating_array.nonzero()] = 1
for i in range(user_average_array.shape[0]):
    init_UI[i] = init_UI[i] * (user_average_array[i]-0.001) 
user_average_array = init_UI
ratingWuserAvg_array = rating_array - user_average_array
ratingWuserAvg_matrix=sparse.csr_matrix(ratingWuserAvg_array)

#### Split to get rtrain-UI matrix and valid and test.. item_index_matrix_train

In [19]:
rtrain_implicit, rvalid_implicit, rtest_implicit, rtrain_userAvg_implicit, rvalid_userAvg_implicit, rtest_userAvg_implicit, nonzero_index, rtime, item_idx_matrix_train_implicit,item_idx_matrix_valid_implicit, item_idx_matrix_test_implicit = time_ordered_splitModified(rating_matrix=rating_matrix, ratingWuserAvg_matrix=ratingWuserAvg_matrix, timestamp_matrix=timestamp_matrix,
                                                                     ratio=[0.5,0.2,0.3],
                                                                     implicit=True,
                                                                     remove_empty=False, threshold=3,sampling=False, 
                                                                     sampling_ratio=0.1, trainSampling=0.95)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  del sys.path[0]
100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 5045.36it/s]


In [20]:
rtrain, rvalid, rtest, rtrain_userAvg, rvalid_userAvg, rtest_userAvg, nonzero_index, rtime, item_idx_matrix_train,item_idx_matrix_valid, item_idx_matrix_test = time_ordered_splitModified(rating_matrix=rating_matrix, ratingWuserAvg_matrix=ratingWuserAvg_matrix, timestamp_matrix=timestamp_matrix,
                                                                     ratio=[0.5,0.2,0.3],
                                                                     implicit=False,
                                                                     remove_empty=False, threshold=3,
                                                                     sampling=False, sampling_ratio=0.1, 
                                                                     trainSampling=0.95)  

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 3957.59it/s]


#### Get df shrink to df_train for rtrain

In [21]:
#Get the list of row index for the training set 
lst_train = get_corpus_idx_list(df, item_idx_matrix_train)

# Get the training dataframe from the original dataframe
df_train = df.loc[lst_train]

#Resetting the index of the train data
df_train = df_train.reset_index(drop=True)

df_train.shape

100%|██████████████████████████████████████████████████████████████████████████████| 6099/6099 [01:25<00:00, 71.74it/s]


(91381, 43)

In [22]:
pd.set_option('display.max_columns', None)
df_train.head(5)

,Day,Month,Unnamed: 0.1,Unnamed: 0_x,Unnamed: 0_y,Updated,Year,alias,business_id,business_stars,categories,coordinates,date,display_phone,distance,friend_count,ghost,image_url,img_dsc,img_url,is_closed,location,name,nr,phone,photo_count,price,review_count_x,review_count_y,review_date,review_id,review_language,review_stars,review_text,transactions,ufc,url,user_id,user_loc,vote_count,business_num_id,user_num_id,timestamp
0,13,1,628892,628892,8039,False,2017,bobbette-and-belle-toronto,vcxvQyAggPqxcHwvJXvjGg,4.0,"[{'alias': 'catering', 'title': 'Caterers'}, {...","{'latitude': 43.66201, 'longitude': -79.33476}",2017-01-13,+1 416-466-8800,7046.832404,4,False,https://s3-media3.fl.yelpcdn.com/bphoto/7DRlzP...,[],[],False,"{'address1': '1121 Queen Street E', 'address2'...",Bobbette & Belle,False,1.416467e+10,13.0,$$,78,154,"['1', '13', '2017']",9e3cmOBflFYd98XZVN8YXQ,en,5.0,Food and drink: Loved the alfajores (mini lemo...,[],"[0, 0, 0]",https://www.yelp.com/biz/bobbette-and-belle-to...,--BumyUHiO_7YsHurb9Hkw,"Old Toronto, Toronto, Canada",0.0,3724,0,1.484284e+09
1,13,1,582871,582871,15015,False,2017,pai-northern-thai-kitchen-toronto-5,r_BrIgzYcwo1NAuG9dLbpg,4.5,"[{'alias': 'thai', 'title': 'Thai'}]","{'latitude': 43.647866, 'longitude': -79.3886415}",2017-01-13,+1 416-901-4724,4202.070047,4,False,https://s3-media2.fl.yelpcdn.com/bphoto/KzTHwC...,[],[],False,"{'address1': '18 Duncan Street', 'address2': '...",Pai Northern Thai Kitchen,False,1.416901e+10,13.0,$$,78,2144,"['1', '13', '2017']",El28CtPV5fnMXyofHoGhoA,en,4.0,Food and drink: One of my favourite places for...,[],"[0, 0, 0]",https://www.yelp.com/biz/pai-northern-thai-kit...,--BumyUHiO_7YsHurb9Hkw,"Old Toronto, Toronto, Canada",0.0,3467,0,1.484284e+09
2,23,1,437873,437873,10150,False,2017,college-falafel-toronto,xsl-d_opm3AU5H2Z-im33g,4.0,"[{'alias': 'mideastern', 'title': 'Middle East...","{'latitude': 43.65455, 'longitude': -79.42289}",2017-01-23,+1 416-532-8698,4471.118815,4,False,https://s3-media1.fl.yelpcdn.com/bphoto/NjObd5...,[],[],False,"{'address1': '450 Ossington Ave', 'address2': ...",College Falafel,False,1.416533e+10,13.0,$,78,86,"['1', '23', '2017']",jq-96Xeg9ilhl_jxyN49PA,en,4.0,"Food: The falafel sandwich was great - fresh, ...",[],"[0, 0, 0]",https://www.yelp.com/biz/college-falafel-toron...,--BumyUHiO_7YsHurb9Hkw,"Old Toronto, Toronto, Canada",0.0,3863,0,1.485148e+09
3,23,1,385263,385263,2748,False,2017,bar-raval-toronto,41o1FUbCYKJv2djtnlkzlg,4.0,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...","{'latitude': 43.6559438482769, 'longitude': -7...",2017-01-23,+1 647-344-8001,3530.003000,4,False,https://s3-media2.fl.yelpcdn.com/bphoto/Bx6Tl9...,[],[],False,"{'address1': '505 College Street', 'address2':...",Bar Raval,False,1.647345e+10,13.0,$$$,78,228,"['1', '23', '2017']",htHfoKD30p9sgQoFwye9jQ,en,3.0,Food and drink: Went for cocktails only; I had...,[],"[0, 0, 0]",https://www.yelp.com/biz/bar-raval-toronto?adj...,--BumyUHiO_7YsHurb9Hkw,"Old Toronto, Toronto, Canada",0.0,317,0,1.485148e+09
4,23,1,88984,88984,11232,False,2017,bar-isabel-toronto,q5xrVJ4kivx_yEfJeOKNYQ,4.0,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...","{'latitude': 43.65463, 'longitude': -79.42075}",2017-01-23,+1 416-532-2222,4230.791736,4,False,https://s3-media1.fl.yelpcdn.com/bphoto/Ne8Ykv...,[],[],False,"{'address1': '797 College Street', 'address2':...",Bar Isabel,False,1.416532e+10,13.0,$$$,78,379,"['1', '23', '2017']",jIgYSZAWS0TX4097BN1u8g,en,4.0,"Food and drink: As a vegetarian, the options a...",[],"[0, 0, 0]",https://www.yelp.com/biz/bar-isabel-toronto?ad...,--BumyUHiO_7YsHurb9Hkw,"Old Toronto, Toronto, Canada",0.0,3391,0,1.485148e+09


#### If using term frequency only to compute corpus and X(review vs. terms) CounterVectorizer

In [288]:
# The entire corpus
corpus = preprocess(df_train)
# X row: df_train row, column: key words frequency 
# When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold
cv=CountVectorizer(max_df=0.9,stop_words=stop_words, max_features=5000, ngram_range=(1,1))
X=cv.fit_transform(corpus)

100%|██████████████████████████████████████████████████████████████████████████| 91381/91381 [00:31<00:00, 2919.11it/s]


##### If using TD-IDF to compute corpus and X (business vs. terms) TfIdfVectorizer

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
corpus = preprocess(df_train)

100%|██████████████████████████████████████████████████████████████████████████| 91381/91381 [00:33<00:00, 2716.39it/s]


In [25]:
#Creating a dictionary to store business: review text
dict_text = {}
for i in range(len(corpus)):
    if df_train['business_num_id'][i] not in dict_text:
        dict_text[df_train['business_num_id'][i]] = corpus[i]
    else:
        temp = dict_text[df_train['business_num_id'][i]]
        temp = temp + corpus[i]
        dict_text[df_train['business_num_id'][i]] = temp

In [26]:
#Create a list for the review text, where the row dimension = total business ids
list_text = []
for key in range(0,max(list(dict_text.keys()))+1) :
    if key not in dict_text.keys():
        list_text.extend([""])
    else:
        list_text.extend([dict_text[key]])

In [27]:
#Get the X vector, where dimension is #business vs #terms
vectorizer = TfidfVectorizer(max_df=0.9,stop_words=stop_words, max_features=5000, ngram_range=(1,1))
X_cleaned = vectorizer.fit_transform(list_text).toarray()
X_cleaned_sparse = csr_matrix(X_cleaned)

## User-rating KNN

### 1. With ratings that subtracts user average rating, cosine similarity

In [162]:
#UU similarity, using cosine similarity
similarity_1 = train(rtrain)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
UI_predictionScore_Explicit = predict(rtrain_userAvg, 100, similarity_1, item_similarity_en= False)
UI_predict_Explicit = prediction(UI_predictionScore_Explicit, 50, rtrain_userAvg)
#user_item_res1 = evaluate(user_item_predict1, rvalid_userAvg)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 4583.52it/s]


In [163]:
UI_predict_Explicit[0]

array([2397., 1812., 3393., 3591., 2577.,  102.,  360., 2597., 3467.,
       2006., 1366.,  421., 2875., 3813., 3522., 3795.,  379.,  587.,
        588., 2713., 1047.,  881.,  747.,  621.,  914., 2277., 2540.,
       2527., 1050., 2460., 2801.,  331., 3140., 3613., 2660., 3614.,
       2896.,  307., 1711., 1579., 1902., 3419.,  211.,  828.,  109.,
       3032., 3945., 2510., 1384., 1074.])

### 2. Implicit User-rating

In [268]:
#UU similarity, using cosine similarity
similarity_2 = train(rtrain_implicit)
UI_predictionScore_Implicit = predict(rtrain_implicit, 100, similarity_2, item_similarity_en= False)
UI_predict_Implicit = prediction(UI_predictionScore_Implicit, 50, rtrain_implicit)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 4683.62it/s]


In [269]:
UI_predict_Implicit[0]

array([3637., 2397., 2234.,  747., 2741.,  102., 2875., 3032.,  588.,
       2713., 2726., 1711., 2930., 2517., 1440., 2577., 2880., 3615.,
       2560., 1366., 1790., 2524., 2597.,  621., 1880.,  295.,  833.,
       3160., 1083., 3155.,  768., 2896., 2841., 1741., 2732., 3933.,
        683., 2219., 2450.,  164., 3204.,  587.,  815.,  722., 3390.,
       1356., 2038., 3326., 2571., 1862.])

### 3. Implicit similarity, Explicit user-rating combination


In [166]:
similarity_3 = train(rtrain_implicit)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
UI_predictionScore_IECombined = predict(rtrain, 100, similarity_3, item_similarity_en= False)
UI_predict_IECombined = prediction(UI_predictionScore_IECombined, 50, rtrain)
# user_item_res1 = evaluate(UI_predict_IECombined, rvalid)
# user_item_res1

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 4460.23it/s]


### 4. NO NEED Something random, to be filled

In [167]:
similarity_4 = train(rtrain)
UI_predictionScore_random1 = predict(rtrain_implicit, 100, similarity_4, item_similarity_en= False)
UI_predict_random1 = prediction(UI_predictionScore_random1, 50, rtrain_implicit)
# user_item_res1 = evaluate(UI_predict_IECombined, rvalid)
# user_item_res1

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 4622.96it/s]


### 5. something random, to be filled 2

In [168]:
userVisitMatrix = getImplicitMatrix(rtrain)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  This is separate from the ipykernel package so we can avoid doing imports until


In [169]:
similarity1 = train(rtrain)
similarity2 = train(rtrain_implicit)
similarity3 =train(userVisitMatrix)
similarity4 = train(rtrain_userAvg)

In [170]:
UI_predictionScore_max = predictUU(rtrain, 100, 'max', similarity1, similarity2, similarity3, similarity4)
UI_predict_max = prediction(UI_predictionScore_max, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 4381.99it/s]


### 6. IU, Explicit

In [178]:
#UU similarity, using cosine similarity
similarity_6 = train(rtrain.T)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
IU_predictionScore_Explicit = predict(rtrain, 100, similarity_6, item_similarity_en= True)
IU_predict_Explicit = prediction(IU_predictionScore_Explicit, 50, rtrain)
#user_item_res1 = evaluate(user_item_predict1, rvalid_userAvg)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 3850.54it/s]


In [180]:
IU_predict_Explicit[0]

array([3637., 3467.,  747., 2234., 2397., 1887., 3160.,  144., 1511.,
       1246., 1510.,  902., 2193., 1440.,  683., 2841., 1138., 2076.,
       3393.,  881.,  492., 2142., 1536., 3297., 1109., 1334., 3010.,
       3613., 1619., 2006., 3214., 2460.,  102., 3615., 2913., 3415.,
       3176., 2597., 3959.,  621., 2724.,  316.,  380., 3371., 2219.,
        864., 3401.,  331., 3324.,  816.])

### 7. Item_based TF-IDF

In [316]:
IK_MATRIX = X_cleaned_sparse
I_I_similarity = train(IK_MATRIX)
item_based_prediction_score4 = predict(rtrain, 10, I_I_similarity, item_similarity_en= True)
#for each restuarant top50 users 
Item_predict_tfidf = prediction(item_based_prediction_score4, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 4052.49it/s]


In [318]:
Item_predict_tfidf[0]

array([1126.,   24.,   14., 3711., 1211., 1945., 2822., 2148.,  993.,
       1554., 1866., 1334., 3390., 2969.,  206.,  677., 3615.,  146.,
       1626.,  714., 2625., 3948., 1194.,  961., 1915., 2389., 1812.,
       1007.,  665., 2248., 3036., 3607., 2865.,  628.,  325., 3801.,
       2052., 2287., 2446., 1382., 3854., 2624.,  360., 3411., 1265.,
       3710., 3819., 3184., 3679., 3311.])

### Find restaurant recommendation within set location radius

#### get the popular items in three ways
1. avg stars
2. number of reviews
3. percentage liked

In [37]:
# dff = df.copy()
dff_popular = df.copy()
dff_popular = dff_popular.sort_values(by=["review_count_y"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
popular_list = dff_popular["business_num_id"].tolist()


In [486]:
def get_three_popularity_matrix(df_original,rtrain):
    # get the list of popular items by ranking the number of reviews
    dff_popular = df_original.copy()
    dff_popular = dff_popular.sort_values(by=["review_count_y"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_num_of_reviews = dff_popular["business_num_id"].tolist()
    
    # get the list of popular items by ranking average rating score
    dff_popular_rating = df_original.copy()
    dff_popular_rating = dff_popular.sort_values(by=["business_stars"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_avg_stars = dff_popular_rating["business_num_id"].tolist()
    
    
    # get the popularity items by using the percentage liked method(number of liked items / total items)
    numUsers = rtrain.shape[0]
    numItems = rtrain.shape[1]
    
    predictionMatrix = np.zeros((numUsers , numItems))

    # Define function for converting 1-5 rating to 0/1 (like / don't like)
    vf = np.vectorize(lambda x: 1 if x >= 4 else 0)
    rtrain_array = rtrain.toarray()
    # For every item calculate the number of people liked (4-5) divided by the number of people that rated
    itemPopularity = np.zeros((numItems))
    for item in range(numItems):
        numOfUsersRated = len(rtrain_array[:, item].nonzero()[0])
        numOfUsersLiked = len(vf(rtrain_array[:, item]).nonzero()[0])
#         if numOfUsersRated == 0:
        # set a threshold to filter out restaurants with very few reviews
        if numOfUsersRated <= 100:
            itemPopularity[item] = 0
        else:
            itemPopularity[item] = numOfUsersLiked/numOfUsersRated
    popular_list_liked_ratio = itemPopularity.argsort()
    
    return np.asarray(popular_list_num_of_reviews),np.asarray(popular_list_avg_stars),popular_list_liked_ratio

In [473]:
# get number of users and number of items
numUsers = rtrain.shape[0]
numItems = rtrain.shape[1]

In [487]:
# get the 1d array of avg. stars, number of reviews and percentage liked ratio for the three popular metric
popular_list_num_of_reviews,popular_list_avg_stars,popular_list_liked_ratio = get_three_popularity_matrix(df,rtrain)
matrix_popular_list_num_of_reviews = np.tile(popular_list_num_of_reviews,(numUsers,1))
matrix_popular_list_avg_stars = np.tile(popular_list_avg_stars,(numUsers,1))
matrix_popular_list_liked_ratio = np.tile(popular_list_liked_ratio,(numUsers,1))

#### small analysis on the three popularity methods

In [41]:
popular_list_num_of_reviews_result = evaluate(matrix_popular_list_num_of_reviews, rvalid)
popular_list_num_of_reviews_result

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:00<00:00, 7044.63it/s]


{'MAP@5': (0.0205, 0.0022),
 'MAP@10': (0.0179, 0.0016),
 'MAP@15': (0.0165, 0.0013),
 'MAP@20': (0.0157, 0.0011),
 'MAP@50': (0.013, 0.0007)}

In [477]:
popular_list_avg_stars_result = evaluate(matrix_popular_list_avg_stars, rvalid)
popular_list_avg_stars_result

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:00<00:00, 7144.06it/s]


{'MAP@5': (0.0001, 0.0001),
 'MAP@10': (0.0002, 0.0001),
 'MAP@15': (0.0002, 0.0001),
 'MAP@20': (0.0003, 0.0001),
 'MAP@50': (0.0003, 0.0001)}

In [488]:
popular_list_liked_ratio_result = evaluate(matrix_popular_list_liked_ratio, rvalid)
popular_list_liked_ratio_result

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:00<00:00, 7237.16it/s]


{'MAP@5': (0.0013, 0.0005),
 'MAP@10': (0.0012, 0.0004),
 'MAP@15': (0.0012, 0.0003),
 'MAP@20': (0.0012, 0.0003),
 'MAP@50': (0.0013, 0.0002)}

In [489]:
popular_list_liked_ratio_result["MAP@10"][0]

0.0012

In [490]:
data = [["Numer of Reviews",popular_list_num_of_reviews_result["MAP@10"][0]],\
        ["Average Stars",popular_list_avg_stars_result["MAP@10"][0]],\
        ["Liked Ratio(threshold@0)",0.0004],\
        ["Liked Ratio(threshold@50)",0.0007,],\
        ["Liked Ratio(threshold@100)",popular_list_liked_ratio_result["MAP@10"][0]]\
        ]
df_small_analysis = pd.DataFrame(data, columns = ['Method', 'Map@10']) 

In [491]:
df_small_analysis

,Method,Map@10
0,Numer of Reviews,0.0179
1,Average Stars,0.0002
2,Liked Ratio(threshold@0),0.0004
3,Liked Ratio(threshold@50),0.0007
4,Liked Ratio(threshold@100),0.0012


popular_list_liked_ratio_result --> filter out items with reviews less than 100. Threshold cannot be larger since otherwise it could not differentiate with the "number of reviews" method. It could not be 100 and less since the precision is too bad, it could not represent popular items.

### analyze and visualize locations of the popular items

In [44]:
pip install gmplot

  Created wheel for gmplot: filename=gmplot-1.2.0-cp37-none-any.whl size=143770 sha256=99ddcb5169eae91ceb60661ce76598c8fb73be4e53beb3df67e342da1b9b3bd4
  Stored in directory: C:\Users\songya25\AppData\Local\pip\Cache\wheels\81\6a\76\4dd6a7cc310ba765894159ee84871e8cd55221d82ef14b81a1
Successfully built gmplot
Note: you may need to restart the kernel to use updated packages.


In [45]:
import matplotlib.pyplot as plt
import json 
import gmplot 
  

In [46]:

# GoogleMapPlotter return Map object 
# Pass the center latitude and 
# center longitude 
gmap1 = gmplot.GoogleMapPlotter(43.663149, -79.395718, 13 ) 
  
# Pass the absolute path 
gmap1.draw( "..\\data\\map11.html" ) 

# empty map
# file:///W:/Capstone/Tina/iNAGO_RecSys/data/map11.html

In [48]:
popular_list_liked_ratio

array([   0, 2636, 2637, ..., 3234, 3351, 1812], dtype=int64)

In [50]:
dff = df.copy()

In [56]:
array_coordinates = dff.coordinates.values

In [57]:
lat = []
long = []
for item in range(array_coordinates.shape[0]):
    coordinateDict = yaml.safe_load(array_coordinates[item])
    lat.append(coordinateDict["latitude"])
    long.append(coordinateDict["longitude"])
    
latitude_list = lat
  
longitude_list = long
  
gmap4 = gmplot.GoogleMapPlotter(43.663149, -79.395718, 13 ) 
  
# heatmap plot heating Type 
# points on the Google map 
gmap4.heatmap( latitude_list, longitude_list ) 
  
gmap4.draw( "..\\data\\toronto_mapping.html" ) 
# file:///W:/Capstone/Tina/iNAGO_RecSys/data/toronto_mapping.html

### Recommend 3 lists for three locations by using "popular_list_liked_ratio" method

In [444]:
# either a list or 
def geographical_dist(prediction_matrix,intersection,user_id, bus_indexRange):
    lst = []
    length = 0
    if isinstance(prediction_matrix, list):
        length = len(prediction_matrix)
    else:
        length = prediction_matrix[user_id].shape[0]
    for j in range(length):
        if isinstance(prediction_matrix, list):
            coordinateDict = yaml.safe_load(dff[dff["business_num_id"] == prediction_matrix[j]].iloc[0].coordinates)
        else:    
            coordinateDict = yaml.safe_load(dff[dff["business_num_id"] == prediction_matrix[user_id][j]].iloc[0].coordinates)
        test_point = Point(coordinateDict['latitude'],coordinateDict['longitude'])

        result = distance.distance(intersection,test_point).kilometers
        if result<=0.6:
            if isinstance(prediction_matrix, list):
                lst.append(prediction_matrix[j])
            else:
                lst.append(prediction_matrix[user_id][j])
        lst = lst[0:bus_indexRange]
    return lst

In [436]:
# # either a list or 
# def geographical_dist(prediction_matrix,intersection,user_id, bus_indexRange):
#     list = []
#     length = 0
#     if isinstance(prediction_matrix, type(list)):
#         length = len(popular_list)
#     else:
#         length = prediction_matrix[user_id].shape[0]
#     for j in range(length):
#         if isinstance(prediction_matrix, type(list)):
#             coordinateDict = yaml.safe_load(dff[dff["business_num_id"] == popular_list[j]].iloc[0].coordinates)
#         else:    
#             coordinateDict = yaml.safe_load(dff[dff["business_num_id"] == prediction_matrix[user_id][j]].iloc[0].coordinates)
#         test_point = Point(coordinateDict['latitude'],coordinateDict['longitude'])

#         result = distance.distance(intersection,test_point).kilometers
#         if result<=0.5:
#             if isinstance(prediction_matrix, type(list)):
#                 list.append(popular_list[j])
#             else:
#                 list.append(prediction_matrix[user_id][j])
#         list = list[0:bus_indexRange]
#     return list

In [59]:
# three locations for user input
yonge_and_finch = Point("43.779824, -79.415665")
bloor_and_bathurst = Point("43.665194,-79.411208")
queen_and_spadina = Point("43.648772,-79.396259")

In [60]:
# three locations for user recommendation
bloor_and_yonge = Point("43.670409,-79.386814")
dundas_and_yonge = Point("43.6561,-79.3802")
spadina_and_dundas = Point("43.653004,-79.398082")

In [61]:
# #
# bay_and_queens = Point("43.6518,-79.3802")
# king_and_jarvis = Point("43.650577,-79.371887")

# yonge_and_eglinton = Point("43.7064,-79.3986")

In [62]:
# list of percentage liked ratio 
list_popular_liked_ratio = popular_list_liked_ratio.tolist()

#### 10 items for each location for user input

In [443]:
yonge_and_finch_list = geographical_dist(list_popular_liked_ratio,yonge_and_finch,0, 10)
yonge_and_finch_list

[2655, 2682, 2584, 2931, 2807, 2182, 2063, 2354, 2360, 2255]

In [442]:
bloor_and_bathurst_list = geographical_dist(list_popular_liked_ratio,bloor_and_bathurst,0, 10)
bloor_and_bathurst_list

[2678, 2482, 2490, 2497, 2571, 2567, 2731, 2910, 2963, 2946]

In [441]:
queen_and_spadina_list = geographical_dist(list_popular_liked_ratio,queen_and_spadina,0, 10)
queen_and_spadina_list

[2637, 2643, 2644, 2648, 2654, 2662, 2649, 2632, 2608, 2624]

In [67]:
df_location = df.copy()

#### list 1-3 demo for user input

In [446]:
df_location_yonge_and_finch=df_location[df_location["business_num_id"].isin(yonge_and_finch_list)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name","categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})
df_location_yonge_and_finch

,business_num_id,name,categories,business_stars,review_count
1802,2063,Pastel Creperies & Dessert House,"[{'alias': 'creperies', 'title': 'Creperies'},...",4.0,316
35939,2354,Cafe Bene,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",3.5,84
41602,2931,Hot Star Large Fried Chicken,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",3.5,52
52211,2584,Sansotei Ramen,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.0,94
64065,2255,Jung Soo Nae Restaurant,"[{'alias': 'korean', 'title': 'Korean'}]",3.5,46
83975,2682,Burrito Place,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.0,61
113360,2807,Daisy Nails and Spa,"[{'alias': 'othersalons', 'title': 'Nail Salon...",4.0,92
139434,2655,Sang-Ji Fried Bao,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",4.0,93
188960,2182,Go Topoki,"[{'alias': 'korean', 'title': 'Korean'}]",4.0,90
192364,2360,Chicken in the Kitchen,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",3.5,9


In [447]:
df_location_bloor_and_bathurst_list=df_location[df_location["business_num_id"].isin(bloor_and_bathurst_list)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name","categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})
df_location_bloor_and_bathurst_list

,business_num_id,name,categories,business_stars,review_count
6915,2963,BMV Books,"[{'alias': 'bookstores', 'title': 'Bookstores'...",4.0,140
99945,2571,Hodo Kwaja,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,188
105714,2946,Jin Dal Lae,"[{'alias': 'korean', 'title': 'Korean'}]",4.5,42
122426,2567,Fuzz Wax Bar,"[{'alias': 'waxing', 'title': 'Waxing'}, {'ali...",4.0,40
126487,2910,PAT Central Market,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.0,83
133705,2490,Sora Beauty Salon,"[{'alias': 'hair', 'title': 'Hair Salons'}]",4.0,59
134665,2731,KINTON RAMEN,"[{'alias': 'ramen', 'title': 'Ramen'}]",3.5,111
154335,2482,Freezone JaYuGongGan Karaoke,"[{'alias': 'karaoke', 'title': 'Karaoke'}]",3.5,27
181795,2678,El Pocho Antojitos Bar,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,47
195825,2497,El Furniture Warehouse,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,292


In [448]:
df_location_queen_and_spadina_list=df_location[df_location["business_num_id"].isin(queen_and_spadina_list)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name","categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})
df_location_queen_and_spadina_list

,business_num_id,name,categories,business_stars,review_count
50478,2644,LCBO,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...",4.0,31
71107,2643,Gabby's Grill and Bar,"[{'alias': 'newcanadian', 'title': 'Canadian (...",2.5,120
74469,2637,Carver,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,45
99038,2654,R&D Restaurant,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",3.5,216
114661,2624,La Palette,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.0,215
130424,2608,The Porch Toronto,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,125
177237,2632,The Fifth & Terrace,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.0,55
181310,2662,Juicy Dumpling,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,73
187596,2648,b.good,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",3.5,36
189428,2649,Hong Kong Bistro Cafe,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.0,87


### Produce the list of restaurants close to the set location

Initial setting, set for user Index

In [171]:
# three locations for user recommendation
intersection = [bloor_and_yonge,dundas_and_yonge,spadina_and_dundas]
businessIndexRange = 3
prediction_matrix = {}


# manually enter this number
userIndex = 0


In [270]:
#First list
Final_UI_predict_Explicit = geographical_dist(UI_predict_Explicit,intersection[0],userIndex,businessIndexRange)

#Second list
Final_UI_predict_Implicit = geographical_dist(UI_predict_Implicit,intersection[0],userIndex,businessIndexRange)

#Third list
Final_UI_predict_IECombined = geographical_dist(UI_predict_IECombined,intersection[0],userIndex,businessIndexRange)

#Fourth lsit
Final_UI_predict_popular = geographical_dist(popular_list,intersection[0],userIndex,businessIndexRange)

#Five list
Final_UI_predict_max = geographical_dist(UI_predict_max,intersection[0],userIndex,businessIndexRange)



In [271]:
Final_UI_predict_popular

[1243, 1238, 1541]

In [222]:
Final_IU_predict_Explicit = geographical_dist(IU_predict_Explicit,intersection[0],0,businessIndexRange)
Final_IU_predict_Explicit

[2913.0, 621.0]

In [224]:
Final_UI_predict_popular

[1243, 1238, 1541]

In [319]:
metric = [UI_predict_Implicit, popular_list, UI_predict_IECombined, IU_predict_Explicit, Item_predict_tfidf]

In [327]:
res = [[] for i in range(len(metric))]
for i in range(len(intersection)):
    for j in range(len(metric)):
        temp = geographical_dist(metric[j],intersection[i],userIndex,businessIndexRange)
        res[j] += temp
#     #First list
#     temp_UI_predict_Implicit = geographical_dist(UI_predict_Implicit,intersection[i],userIndex,businessIndexRange)
#     list_UI_predict_Implicit += temp_UI_predict_Implicit
    
#     #Second list
#     temp_UI_predict_popular = geographical_dist(popular_list,intersection[i],userIndex,businessIndexRange)
#     list_UI_predict_popular += temp_UI_predict_popular
    
#     #Third list
#     temp_UI_predict_IECombined = geographical_dist(UI_predict_IECombined,intersection[i],userIndex,businessIndexRange)
#     list_UI_predict_IECombined += temp_UI_predict_IECombined
    
#     #Fourth list
#     temp_IU_predict_Explicit = geographical_dist(IU_predict_Explicit,intersection[i],userIndex,businessIndexRange)
#     list_IU_predict_Explicit += temp_IU_predict_Explicit
    
#     temp_UI_predict_Explicit = geographical_dist(UI_predict_Explicit,intersection[i],userIndex,businessIndexRange)
#     list_UI_predict_Explicit += temp_UI_predict_Explicit 

In [328]:
res

[[2560.0, 621.0, 3326.0, 2741.0, 2713.0, 3615.0, 2726.0, 2517.0, 3155.0],
 [1243, 1238, 2730, 1787, 1790, 41, 2926, 1480, 1536],
 [2560.0, 621.0, 3025.0, 2741.0, 1790.0, 2713.0, 964.0, 2517.0, 2726.0],
 [2913.0, 621.0, 881.0, 3010.0, 3214.0, 1887.0, 1246.0, 2193.0],
 [14.0, 3036.0, 3615.0, 1866.0, 1812.0, 628.0]]

In [454]:
res_final = []
for element in range(len(res)):
    dic = {}
    for i in range(len(res[element])):
        if isinstance(metric[element], list):
            dic[metric[element].index(res[element][i])] = res[element][i]
        else:
            dic[metric[element][userIndex].tolist().index(res[element][i])] = res[element][i]
    temp = []
    for j in sorted(dic.keys()):
        temp.append(dic[j])
    res_final.append(temp[0:3])

In [458]:
res_final

[[2741.0, 2713.0, 2726.0],
 [1787, 2926, 1480],
 [964.0, 2741.0, 2517.0],
 [1887.0, 1246.0, 2193.0],
 [14.0, 1866.0, 3615.0]]

## Match restaurant information according to business_num_id

In [ ]:
def constructResDictionary(userIndex, busIndexRange, UI_prediction):
    dictionaryToConstruct = {}
    #Loop through the number of businesses 
    for busIndex in range(busIndexRange):
        businessSeries = df[df["business_num_id"] == UI_prediction[busIndex]].iloc[0]
        busName = businessSeries['name']
        address_generator = (str(w) for w in yaml.safe_load(businessSeries.location)['display_address'])
        busLocation = ', '.join(address_generator)
        bus_Price = businessSeries.price
        busStars = businessSeries.business_stars
        busReviewCount = businessSeries.review_count_y 
        category_generator = (str(s) for s in [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)])
        #busCategories = [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)]
        busCategories = ', '.join(category_generator)
        #Now add the restaurant to the dictionary
        dictionaryToConstruct[busName] = {'Address': busLocation,\
                                'Price': bus_Price,\
                                 'Star': busStars, \
                                 'Review Count': busReviewCount, \
                                 'Category': busCategories}
    return dictionaryToConstruct

In [39]:
#Trying to recommend for user 0 for now 


dict1_ExplicitRecommend = {}
dict2_ImplicitRecommend = {}
dict3_EICombineRecommend = {}
dict4_PopularityRecommend = {}
dict5_maxRecommend = {}
#User predict [user index] [item index]
#Add a for loop for the top recommended items

dict1_ExplicitRecommend = constructResDictionary(userIndex, businessIndexRange, UI_predict_Explicit)
dict2_ImplicitRecommend = constructResDictionary(userIndex, businessIndexRange, UI_predict_Implicit)
dict3_EICombineRecommend = constructResDictionary(userIndex, businessIndexRange, UI_predict_IECombined)
dict4_PopularityRecommend = constructResDictionary(userIndex, businessIndexRange, UI_predict_popular)
dict5_maxRecommend = constructResDictionary(userIndex, businessIndexRange, UI_predict_max)

#Say we are recommending 5 restaurants for now
# for busIndex in range(5):
#     businessSeries = df[df["business_num_id"] == UI_predict_Explicit[0][busIndex]].iloc[0]
#     busName = businessSeries['name']
#     address_generator = (str(w) for w in yaml.safe_load(businessSeries.location)['display_address'])
#     busLocation = ', '.join(address_generator)
#     bus_Price = businessSeries.price
#     busStars = businessSeries.business_stars
#     busReviewCount = businessSeries.review_count_y 
#     category_generator = (str(s) for s in [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)])
#     #busCategories = [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)]
#     busCategories = ', '.join(category_generator)
#     #Now add the restaurant to the dictionary
#     dict1stAlgo[busName] = {'Address': busLocation,\
#                             'Price': bus_Price,\
#                              'Star': busStars, \
#                              'Review Count': busReviewCount, \
#                              'Category': busCategories}

In [40]:
class MultiListbox(Frame):
    
    def __init__(self, master, lists):
        Frame.__init__(self, master)
        self.lists = []
        
        #Loop through the lists
        for l,widthW in lists:
            
            frame = Frame(self); frame.pack(side=LEFT, expand=YES, fill=BOTH)
            
            Label(frame, text=l, borderwidth=1, relief=RAISED).pack(fill=X)
            
            lb = Listbox(frame, width=widthW, borderwidth=0, selectborderwidth=0,
                 relief=FLAT, exportselection=FALSE)
            lb.pack(expand=YES, fill=BOTH)
            
            self.lists.append(lb)
            lb.bind('<B1-Motion>', lambda e, s=self: s._select(e.y))
            lb.bind('<Button-1>', lambda e, s=self: s._select(e.y))
            
            lb.bind('<Leave>', lambda e: 'break')
            
            lb.bind('<B2-Motion>', lambda e, s=self: s._b2motion(e.x, e.y))
            lb.bind('<Button-2>', lambda e, s=self: s._button2(e.x, e.y))
            
        frame = Frame(self); frame.pack(side=LEFT, fill=Y)
        
        Label(frame, borderwidth=1, relief=RAISED).pack(fill=X)
        
        #Setting a scrollbar 
#         sb = Scrollbar(frame, orient=VERTICAL, command=self._scroll)
        
#         sb.pack(expand=YES, fill=Y)
        
#         self.lists[0]['yscrollcommand']=sb.set

    def _select(self, y):
        row = self.lists[0].nearest(y)
        self.selection_clear(0, END)
        self.selection_set(row)
        return 'break'

    def _button2(self, x, y):
        for l in self.lists: l.scan_mark(x, y)
        return 'break'

    def _b2motion(self, x, y):
        for l in self.lists: l.scan_dragto(x, y)
        return 'break'

    def _scroll(self, *args):
        for l in self.lists:
            apply(l.yview, args)

    def curselection(self):
        return self.lists[0].curselection()

    def delete(self, first, last=None):
        for l in self.lists:
            l.delete(first, last)

#     def get(self, first, last=None):
#         result = []
#         for l in self.lists:
#             result.append(l.get(first,last))
#         if last: return apply(map, [None] + result)
#         return result

    def index(self, index):
        self.lists[0].index(index)

    def insert(self, index, *elements):
        #Loop through the elements 
        for element in elements:
            i = 0
            for l in self.lists:
                l.insert(index, element[i])
                i = i + 1

    def size(self):
        return self.lists[0].size()

    def see(self, index):
        for l in self.lists:
            l.see(index)

    def selection_anchor(self, index):
        for l in self.lists:
            l.selection_anchor(index)

    def selection_clear(self, first, last=None):
        for l in self.lists:
            l.selection_clear(first, last)

    def selection_includes(self, index):
        return self.lists[0].selection_includes(index)

    def selection_set(self, first, last=None):
        for l in self.lists:
            l.selection_set(first, last)

In [42]:
if __name__ == '__main__':
    #Initiate a frame or master? 
    tk = Tk()
    Label(tk, text='List of recommended restaurants').pack()
    
    #Creating the multi-listbox object, pass in a tuple of tuple (lists)
    mlb = MultiListbox(tk, (('Explicit User-Ratings', 20), ('Implicit User-Ratings', 20), \
                            ('Explicit Implicit Combined', 20), ('Popularity List', 20), ('Taking Max', 20)))
    
    for index in range(len(dict1_ExplicitRecommend)):
        #First restaurant information to display 
        #Following the restaurants' name 
        restList1 = list(dict1_ExplicitRecommend.keys())[index]
        restList2 = list(dict2_ImplicitRecommend.keys())[index]
        restList3 = list(dict3_EICombineRecommend.keys())[index]
        restList4 = list(dict4_PopularityRecommend.keys())[index]
        restList5 = list(dict5_maxRecommend.keys())[index]
        
        mlb.insert(END, (' ', ' ', ' ', ' ', ' '))
        
        #Inserting the restaurant names
        mlb.insert(END, ('%d: %s' % (index + 1, restList1),'%d: %s' % (index + 1, restList2),
                         '%d: %s' % (index + 1, restList3),'%d: %s' % (index + 1, restList4),
                         '%d: %s' % (index + 1, restList5)))
        
        #Looping through each attribute keys - resinfo
        for resinfo in dict1_ExplicitRecommend.get(restList1).keys():
            restList1Info = resinfo + ':' + str(dict1_ExplicitRecommend.get(restList1).get(resinfo,''))
            restList2Info = resinfo + ':' + str(dict2_ImplicitRecommend.get(restList2).get(resinfo,''))
            restList3Info = resinfo + ':' + str(dict3_EICombineRecommend.get(restList3).get(resinfo,''))
            restList4Info = resinfo + ':' + str(dict4_PopularityRecommend.get(restList4).get(resinfo,''))
            restList5Info = resinfo + ':' + str(dict5_maxRecommend.get(restList5).get(resinfo,''))
            
            mlb.insert(END, (restList1Info, restList2Info, restList3Info, restList4Info, restList5Info))
        
        mlb.insert(END, ('----------------', '----------------', '----------------', '----------------', '----------------'))
    
        
    mlb.pack(expand=YES,fill=BOTH)
    tk.mainloop()

## Implicit

In [ ]:
#UU similarity, using cosine similarity
similarity1 = train(rtrain_implicit)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
user_item_prediction_score1 = predict(rtrain_implicit, 90, similarity1, item_similarity_en= False)
user_item_predict1 = prediction(user_item_prediction_score1, 50, rtrain_implicit)
user_item_res1 = evaluate(user_item_predict1, rvalid_implicit)
user_item_res1

In [ ]:
print(rvalid_implicit.sum())

## Combine implicit and explicit

In [ ]:
#UU similarity, using cosine similarity
similarity1 = train(rtrain_implicit)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
user_item_prediction_score1 = predict(rtrain, 90, similarity1, item_similarity_en= False)
user_item_predict1 = prediction(user_item_prediction_score1, 50, rtrain)
user_item_res1 = evaluate(user_item_predict1, rvalid)
user_item_res1

### 2. With ratings that subtracts user average rating, pearson correlation

In [ ]:
# #UU similarity, using cosine similarity
# similarity2 = train(rtrain_userAvg)
# #get a user-item matrix  UI prediction
# #Predict using UI matrix with ratings in it 
# user_item_prediction_score2 = predict(rtrain_userAvg, 100, similarity2, item_similarity_en= False)
# user_item_predict2 = prediction(user_item_prediction_score2, 50, rtrain_userAvg)
# user_item_res2 = evaluate(user_item_predict2, rvalid_userAvg)
# user_item_res2

### 3. With raw ratings, cosinesimilarity

In [ ]:
#UU similarity
similarity3 = train(rtrain)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
user_item_prediction_score3 = predict(rtrain, 90, similarity3, item_similarity_en= False)
user_item_predict3 = prediction(user_item_prediction_score3, 50, rtrain)
#Check user item prediction score
user_item_res3 = evaluate(user_item_predict3, rvalid)

In [ ]:
user_item_res3

## Item Base KNN using TF-IDF

In [281]:
IK_MATRIX = X_cleaned_sparse

In [282]:
# IK_MATRIX = X_cleaned_sparse
I_I_similarity = train(IK_MATRIX)
item_based_prediction_score4 = predict(rtrain, 10, I_I_similarity, item_similarity_en= True)
#for each restuarant top50 users 
item_based_predict4 = prediction(item_based_prediction_score4, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 4045.27it/s]


In [314]:
item_based_predict4

array([[1126.,   24.,   14., ..., 3184., 3679., 3311.],
       [2497., 3575., 2527., ...,  766., 2970., 3245.],
       [2173., 3368.,   88., ...,  482.,  212., 1533.],
       ...,
       [3813.,  430., 2724., ..., 2576., 3562., 3269.],
       [2411., 2258., 1697., ...,  533., 2727., 2231.],
       [3393.,  245., 3751., ..., 2646., 2645., 2644.]])

In [283]:
item_based_res_TFIDF = evaluate(item_based_predict4, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:00<00:00, 9082.67it/s]


In [284]:
item_based_res_TFIDF

{'MAP@5': (0.0147, 0.0018),
 'MAP@10': (0.0131, 0.0013),
 'MAP@15': (0.0122, 0.0011),
 'MAP@20': (0.0115, 0.0009),
 'MAP@50': (0.0094, 0.0006)}

## Item Base KNN using TF

In [289]:
X

<91381x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 5019585 stored elements in Compressed Sparse Row format>

In [290]:
#3987 items? 5000 key phrase
I_K_matrix = get_I_K(df_train, X, shape = (rating_matrix.shape[1], 5000))
I_I_similarity = train(I_K_matrix)
item_based_prediction_score = predict(rtrain, 10, I_I_similarity, item_similarity_en= True)
#for each restuarant top50 users 
item_based_predict = prediction(item_based_prediction_score, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:01<00:00, 4013.77it/s]


In [291]:
item_based_res = evaluate(item_based_predict, rvalid)
item_based_res

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [00:00<00:00, 8955.86it/s]


{'MAP@5': (0.0172, 0.0019),
 'MAP@10': (0.0154, 0.0014),
 'MAP@15': (0.0143, 0.0012),
 'MAP@20': (0.0134, 0.001),
 'MAP@50': (0.0108, 0.0007)}

## Item based KNN with IC Matrix

In [ ]:
I_C_matrix.shape

In [ ]:
similarity_I_C = train(I_C_matrix)

In [ ]:
user_item_prediction_score_I_C = predict(e
                                         , 90, similarity_I_C, item_similarity_en= True)


In [ ]:
#for each restuarant top50 users 
item_based_predict_I_C = prediction(user_item_prediction_score_I_C, 50, rtrain)

In [ ]:
item_based_res_I_C = evaluate(item_based_predict_I_C, rvalid)

In [ ]:
item_based_res_I_C

## User based KNN with explicit UC Matrix

In [ ]:
I_C_matrix.shape

In [ ]:
rtrain_implicit.shape

In [ ]:
U_C_matrix = rtrain_implicit*I_C_matrix

In [ ]:
similarity_UC = train(U_C_matrix)
user_item_prediction_score_UC = predict(rtrain, 90, similarity_UC, item_similarity_en= False)
user_item_predict_UC = prediction(user_item_prediction_score_UC, 90,rtrain)
user_item_res_UC = evaluate(user_item_predict_UC, rvalid)
user_item_res_UC

In [ ]:
user_item_res_UC

## User based KNN with implicit UC Matrix

In [ ]:
U_C_matrix

In [26]:
def getImplicitMatrix(sparseMatrix, threashold=0):
    temp_matrix = sparse.csr_matrix(sparseMatrix.shape)
    temp_matrix[(sparseMatrix > threashold).nonzero()] = 1
    return temp_matrix

In [ ]:
U_C_matrix_implicit = getImplicitMatrix(U_C_matrix,3)

In [ ]:
U_C_matrix_implicit

In [ ]:
similarity_UC_implicit = train(U_C_matrix_implicit)
user_item_prediction_score_UC_implicit = predict(rtrain, 90, similarity_UC_implicit, item_similarity_en= False)
user_item_predict_UC_implicit = prediction(user_item_prediction_score_UC_implicit, 90,rtrain)
user_item_res_UC_implicit = evaluate(user_item_predict_UC_implicit, rvalid)

In [ ]:
user_item_res_UC_implicit

## Try combining different similarity scores

In [ ]:
#Get a UI matrix if it's not item_similarity based or else IU
def predictUU(matrix_train, k, similarity1, similarity2=None, similarity3=None, similarity4=None, weight1=None, weight2=None, weight3=None, weight4=None, chooseWeigthMethod = None, item_similarity_en = False):
    prediction_scores = []
    
    #inverse to IU matrix
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    #for each user or item, depends UI or IU 
    for user_index in tqdm(range(matrix_train.shape[0])):
    #for user_index in tqdm(range(1)):
        
        numberSimilarMatrix = 0
        # Get user u's prediction scores for all items 
        #Get prediction/similarity score for each user 1*num or user or num of items
        vector_u1 = similarity1[user_index]
        numberSimilarMatrix += 1
        
        if similarity2 is not None:
            vector_u2 = similarity2[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u2 = [0]*len(vector_u1)
            
        if similarity3 is not None:
            vector_u3 = similarity3[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u3 = [0]*len(vector_u1)
            
        if similarity4 is not None:
            vector_u4 = similarity4[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u4 = [0]*len(vector_u1)
        
        vector_u = vector_u1.copy()
            
        #If we are choosing the max, min, or avg or similarity scores
        if chooseWeigthMethod is not None:
            #loop through the user index 
            for item_index in tqdm(range(matrix_train.shape[0])):

                if chooseWeigthMethod == 'max':
                    vector_u[item_index] = max(vector_u1[item_index], vector_u2[item_index], vector_u3[item_index],vector_u4[item_index])
                elif chooseWeigthMethod == 'min':
                    vector_u[item_index] = min(vector_u1[item_index], vector_u2[item_index], vector_u3[item_index],vector_u4[item_index])
                elif chooseWeigthMethod == 'average':
                    vector_u[item_index] = (vector_u1[item_index]+vector_u2[item_index]+vector_u3[item_index]+vector_u4[item_index])/(numberSimilarMatrix)
        
        similar_users = vector_u.argsort()[::-1][1:k+1]
        
        # Get neighbors similarity weights and ratings
        #similar_users_weights = similarity1[user_index][similar_users]
        similar_users_weights = vector_u[similar_users]
        
        #similar_users_weights_sum = np.sum(similar_users_weights)
        #print(similar_users_weights.shape)
        #shape: num of res * k
        similar_users_ratings = matrix_train[similar_users].toarray()
              
        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
        #print(prediction_scores_u)
        
        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
        
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    return res
    #return vector_u

In [ ]:
#UU similarity, using cosine similarity
similarity1 = train(rtrain)
#similarity2 = train(rtrain_userAvg)
similarity2 = None
#similarity3 =train(userVisitMatrix)
similarity3 = None
similarity4 = train(rtrain_implicit)
#similarity4 = None
weight1 = None
weight2=None
weight3=None
#vectorU = predictUU(rtrain_userAvg, 90, similarity1, similarity2, similarity3, weight1, weight2, weight3, item_similarity_en= False)
user_item_prediction_score1 = predictUU(rtrain, 90, similarity1, similarity2, similarity3, similarity4, weight1, weight2, weight3, chooseWeigthMethod = 'max', item_similarity_en= False)
user_item_predict1 = prediction(user_item_prediction_score1, 50, rtrain)
user_item_res1 = evaluate(user_item_predict1, rvalid)

In [ ]:
user_item_res1   #1,4, max rtrain,rvalid

In [ ]:
user_item_res1   #1,2,3,4 max rtrain,rvalid

In [ ]:
user_item_res1  #1,2,3,4, average  rtrain, rvalid 

In [ ]:
user_item_res1  #1,4 average rtrain_implicit 

In [ ]:
user_item_res1  #1,4 average, rtrain_userAvg

In [ ]:
user_item_res1  #1,4 average rtrain, rvalid

In [ ]:
user_item_res1    #1,4 average with userVisitMatrix, rvalid 

In [ ]:
user_item_res1

In [ ]:
user_item_res1 #average 1,4 similarity matrices

In [ ]:
user_item_res1 #average 1,2,4 similarity matrices

In [ ]:
user_item_res1    #average 1-2 similarity matrices

In [ ]:
user_item_res1   #average 3 1-3similarity matrices

In [ ]:
user_item_res1  #average 4, 1-4similarity matrices

In [ ]:
user_item_res1   #min

In [ ]:
user_item_res1 #max

In [ ]:
user_item_res1  #rtrain

In [ ]:
user_item_res1 #rtrain_userAvg

In [ ]:
user_item_prediction_score1.shape

In [ ]:
[3764   36  896 5156 5737  618 2345   11 1706 1554  726  627 5821 3348
  243 1571 1247 5209  203 4856 3754  544 5417 3849 2411 1763  274 5211
   91 2558 4208 4063 4712 3520  457 2900  387 2455 1178 3024 1342 6014
 2823 5848 4691 5099 5109 5545 3987 6069 5794 5572   77 3180 3266 5039
 5904 2325 3628  493 5735   27 4797 4813 3128 5337 4845 3907 3544 2128
  320  134 1985 3470 5944 3188 2438 4850 2630 3912 1245 1674 5986 2360
 5094 3670 5902 1491 3997 3856]

In [ ]:
[3026 4208 5069 5507 1571 3529  544 3090 2538 4797 3527 1558 6079 5736
   13 5209 5363 1866 3640 5684 1062 5213 5757 5417 5986 1466 5902 3109
 4096  464  493  629 3912 3814 1268 3552 2438 5109 1207 4504 5634 4870
 5229  320 4354  785 3167 2416 3128 4540 5457 5420 1241 1674 4850 5140
   15  457  710 1245  942 5147 5164 4314 4346 3648 3180 3103 3189 1357
 4917 3987 5571 2906  210 2489  901 2293 5737  879 2558  564 4310 2119
 5521 5059 2376 3856 1967 4094]

In [ ]:
for number in rtrain_userAvg:
    print(number)

In [ ]:
rtrain_userAvg[0][0].toarray()

In [ ]:
rtrain_userAvg.shape

In [ ]:
def getImplicitMatrix(sparseMatrix, threashold=0):
    temp_matrix = sparse.csr_matrix(sparseMatrix.shape)
    temp_matrix[(sparseMatrix > threashold).nonzero()] = 1
    return temp_matrix

In [ ]:
userVisitMatrix = getImplicitMatrix(rtrain)

In [ ]:
#UU similarity
similarity3 = train(userVisitMatrix)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
user_item_prediction_score3 = predict(userVisitMatrix, 90, similarity3, item_similarity_en= False)
user_item_predict3 = prediction(user_item_prediction_score3, 50, userVisitMatrix)
#Check user item prediction score
user_item_res3 = evaluate(user_item_predict3, rvalid)

In [ ]:
user_item_res3

In [ ]:
rtrain[0].toarray()

In [ ]:
for user_index in tqdm(range(2)):
    print(user_index)

In [ ]:
userVisitMatrix.shape

In [ ]:
rtrain.shape

In [ ]:
max(5,8,9,10)

In [ ]:
stats.mean([1,1,1,1])

In [ ]:
max([5,10,2], [10,5,9], [2,5,13])